## Use Musashi to run little M68K snippets

Fix the cell size to make it usable

In [1]:
from IPython.display import display, HTML
#Enable horizontal scrolling on output
#display(HTML("<style>pre { white-space: pre !important; }</style>"))

# Increase the width
display(HTML("<style>.container { width:80% !important; }</style>"))


Download and compile our customize musashi emulator

In [2]:
import os
! rm -rf Musashi
!git clone https://github.com/mwenge/Musashi.git
os.chdir('Musashi/m68kscratch')
! make
os.chdir('../..')

Cloning into 'Musashi'...
remote: Enumerating objects: 614, done.
remote: Counting objects: 100% (266/266), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 614 (delta 241), reused 231 (delta 231), pack-reused 348 (from 2)
Receiving objects: 100% (614/614), 460.65 KiB | 1.23 MiB/s, done.
Resolving deltas: 100% (398/398), done.
gcc -o  m68kmake  m68kmake.c
./m68kmake

	Musashi v4.60 68000, 68008, 68010, 68EC020, 68020, 68EC030, 68030, 68EC040, 68040 emulator
		Copyright Karl Stenerud (kstenerud@gmail.com)

Generated 1967 opcode handlers from 518 primitives
gcc -Wall -Wextra -pedantic   -c -o m68ksim.o m68ksim.c
gcc -Wall -Wextra -pedantic   -c -o m68kcpu.o m68kcpu.c
gcc -Wall -Wextra -pedantic   -c -o m68kdasm.o m68kdasm.c
gcc -Wall -Wextra -pedantic   -c -o softfloat/softfloat.o softfloat/softfloat.c
gcc -Wall -Wextra -pedantic   -c -o m68kops.o m68kops.c
m68kops.c: In function ‘m68ki_build_opcode_table’:
m68kops.c:36709:13: warning: unused variable ‘cycle_cost’ []

In [164]:
os.chdir('Musashi/m68kscratch')
! make
os.chdir('../..')

gcc -Wall -Wextra -pedantic   -c -o m68ksim.o m68ksim.c
gcc -o m68ksim m68ksim.o m68kcpu.o m68kdasm.o softfloat/softfloat.o m68kops.o -Wall -Wextra -pedantic -lm


Download and compile `rmac` for assembling our little M68K snippets.

In [3]:
!rm -rf rmac
!git clone http://tiddly.mooo.com:5000/rmac/rmac.git
os.chdir('rmac')
!make -j5
os.chdir('..')

Cloning into 'rmac'...
remote: Enumerating objects: 2646, done.
remote: Counting objects: 100% (2646/2646), done.
remote: Compressing objects: 100% (702/702), done.
remote: Total 2646 (delta 2046), reused 2423 (delta 1904), pack-reused 0
Receiving objects: 100% (2646/2646), 882.23 KiB | 940.00 KiB/s, done.
Resolving deltas: 100% (2046/2046), done.
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c kwgen.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c 68kgen.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c debug.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c dsp56k.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c dsp56kgen.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c eagen.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -o dsp56kgen dsp56kgen.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -o 68kgen 68kgen.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -o kwgen kwgen.c
gcc -std=c99 

./dsp56kgen dsp56k.tab <dsp56k.mch >dsp56ktab.h
./kwgen mr <risc.tab >risckw.h
./kwgen mp <6502.tab >6502kw.h
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c rmac.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c symbol.c
./kwgen unary <unary.tab >unarytab.h
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c dirent_lose.c
./kwgen kw <kw.tab >kwtab.h
./kwgen reg65 <6502regs.tab >6502regs.h
./68kgen 68k.tab <68k.mch >68ktab.h
./kwgen reg68 <68kregs.tab >68kregs.h
./kwgen reg56 <56kregs.tab >56kregs.h
./kwgen regrisc <riscregs.tab >riscregs.h
./dsp56kgen dsp56k.tab <dsp56k.mch >dsp56ktab.h
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c expr.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c mach.c
./kwgen mo <op.tab >opkw.h
./kwgen dsp <dsp56k.tab >dsp56kkw.h
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c riscasm.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c sect.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX_

Write, assemble and run a little M68K snippet

In [313]:
program = """
dcb.l 2,0
.org $0000

.text
in_buf equ $4000
scaler equ $4100


; Set up some variables
init:
  lea _web,a6
  move.l #$4000,vp_x
  move.l #$4000,vp_y
  move.l #$4000,vp_z
  move.l vp_x,d3
  move.l vp_y,d4
  move.l vp_z,d5

;***************************************
; Draw the web
;***************************************
drawweb:
  move.l a6,oopss
  move.l (a6),d0
  move.l d0,a1
  lea in_buf+4,a0
  move.l 4(a6),d0
  sub.l d3,d0
  move.l d0,(a0)+    ;Co-ordinates as X, Y, Z 16:16 frax
  move.l 8(a6),d0    ;Combine with camera viewpoint
  sub.l d4,d0
  move.l d0,(a0)+
  move.l 12(a6),d0
  sub.l d5,d0
  tst h2h           ; are we in head-to-head mode?
  beq swebbo
  
  sub.l #$40000,d0  ;hack to avoid floating Flippers (in head to head mode)
  
swebbo:
  move.l d0,(a0)+
  tst h2h           ; are we in head-to-head mode?
  beq dragg
  
  ; in head-to-head mode use xvector as our gpu routine
  lea xvector,a2    ; set the gpu routine to xvector
  bra draaa  

vector:
  move.l d0,a1    ;Get object header
  lea in_buf+4,a0
  move.l 4(a6),d0
  sub.l vp_x,d0
  move.l d0,(a0)+    ;Co-ordinates as X, Y, Z 16:16 frax
  move.l 8(a6),d0    ;Combine with camera viewpoint
  sub.l vp_y,d0
  move.l d0,(a0)+
  move.l 12(a6),d0
  
dra:
  sub.l vp_z,d0
  move.l d0,(a0)+
  
  ; use fastvector for single-player
dragg:
  lea fastvector,a2
  
draaa:
  move 28(a6),d0
druuu:
  and.l #$ff,d0
  move.l d0,24(a1)  ;XY orientation
  move 30(a6),d0
  and.l #$ff,d0
  move.l d0,28(a1)  ;XZ orientation
  move 32(a6),d0
  and.l #$ff,d0
  move.l d0,32(a1)  ;YZ orientation
  
  ; Copy the header we've created at A1 to GPU input RAM.
  move #11,d0    ;copy 48 bytes
xhead:
  move.l (a1)+,(a0)+  ;copy the header to GPU input ram
  dbra d0,xhead
  
  move 40(a6),d0
  and.l #$ff,d0
  move.l d0,(a0)+    ;colour
  move 42(a6),d0
  ext.l d0
  move.l d0,scaler
  move.l a1,oopss+4
  move.l (a6),oopss+8
  
godraa:
  ;move.l #2,gpu_mode
  move.l a2,a0      ; set the gpu routine
  ;jsr gpurun    ;do gpu routine
  ;jsr gpuwait
  ;rts
  stop #$2700

;*********************
; Data
;*********************
.data
_web: 
  dc.w 15,7
  dc.w 7,2,5,3,3,5,2,7    ;circle
  dc.w 2,9,3,11,5,13,7,14
  dc.w 9,14,11,13,13,11,14,9
  dc.w 14,7,13,5,11,3,9,2,7,2,-1
  dc.w 112,96,80,64,48,32,16,0,-16,-32,-48,-64,-80,-96,-112,-128  ;orientation table (angle of an object within a particular lane)
  dc.w 0,$01,$02,$03,$04,$05,$06,07,$07,$6,$05,$04,$03,$02,$01,0


.org $6000
h2h: dc.w 0
vp_x: dc.l $0
vp_y: dc.l $0
vp_z: dc.l $0
fastvector: dc.l $FFFFFFFF
xvector: dc.l $FFFFFFFF
oopss: dc.l 0,0,0,0



"""
open("rmac_prog.s",'w').write(program)

# Assemble and run!
!./rmac/rmac -s ~oall -n -fr -rl -i rmac_prog.s -o tst && ../../Musashi/m68kscratch/m68ksim tst

Legacy mode OFF
rmac_prog.s 37: Warning: unoptimized short branch
rmac_prog.s 44: Warning: unoptimized short branch
rmac_prog.s 48: Warning: unoptimized short branch
  PC   Hex                   Instructions                d0       d1       d2       d3       d4       d5       d6       d7       a0       a1       a2       a3       a4       a5       a6       a7      
E 000: 0000 0000           : ori.b   #$0, D0                                                                                                                                                          
E 004: 0000 0000           : ori.b   #$0, D0                                                                                                                                                          
E 008: 4df9 0000 611c      : lea     $611c.l, A6                                                                                                                                                      
E 00e: 23fc 0000 4000 0000 61a8: mov

In [51]:
program = """
dcb.l 2,0
.org $0000

    ; Point a0 at the position in av1 given by the bank number in imatrix. 
    movea.l av1ref,a0
    move.l  a0,d1
    lea     16(a0),a0
    move    imatrix,d0 ; Get the bank number
    lsl     #2,d0 ; Multiply d0 by 4.
    move.l  (a0,d0.w),d0
    sub.l   #$00900000,d0
    add.l   d1,d0
    movea.l d0,a0
 
    stop #$2700

    imatrix:  dc.w $00 ;.w 0..9 bank number
    av1ref:   dc.l av
    av:
        dc.b  $00,$00,$00,$0A,$00,$90,$B6,$9C ; 0x30
        dc.b  $00,$90,$00,$38,$F4,$35,$2A,$C6 ; 0x38
        dc.b  $00,$90,$02,$00,$00,$90,$18,$56 ; 0x40
        dc.b  $00,$90,$34,$A4,$00,$90,$44,$82 ; 0x48
        dc.b  $00,$90,$57,$76,$00,$90,$61,$D8 ; 0x50
        dc.b  $00,$90,$75,$A6,$00,$90,$88,$06 ; 0x58
        dc.b  $00,$90,$9A,$72,$00,$90,$A6,$B4 ; 0x60
        dc.b  $66,$04,$9E,$A9,$C8,$31,$65,$28 ; 0x68
        dc.b  $8A,$ED,$DC,$28,$08,$74,$24,$27 ; 0x70
        dc.b  $42,$2B,$CE,$06,$B3,$08,$5A,$2B ; 0x78
        dc.b  $27,$34,$55,$82,$D6,$8A,$42,$42 ; 0x80
        dc.b  $DA,$A7,$48,$2F,$4E,$E8,$29,$2B ; 0x88
        dc.b  $F8,$08,$43,$06,$7F,$AF,$24,$98 ; 0x90
        dc.b  $42,$20,$3E,$C5,$FE,$C9,$04,$D6 ; 0x98

"""
open("rmac_prog.s",'w').write(program)

# Assemble and run!
!./rmac/rmac -s ~oall -n -fr -rl -l -i rmac_prog.s -o tst && hexdump tst && ../../Musashi/m68kscratch/m68ksim tst

Legacy mode OFF
0000000 0000 0000 0000 0000 7920 0000 3000 0822
0000010 e841 1000 3930 0000 2e00 48e5 3020 0000
0000020 bc90 9000 0000 81d0 4020 724e 0027 0000
0000030 0000 3400 0000 0a00 9000 9cb6 9000 3800
0000040 35f4 c62a 9000 0002 9000 5618 9000 a434
0000050 9000 8244 9000 7657 9000 d861 9000 a675
0000060 9000 0688 9000 729a 9000 b4a6 0466 a99e
0000070 31c8 2865 ed8a 28dc 7408 2724 2b42 06ce
0000080 08b3 2b5a 3427 8255 8ad6 4242 a7da 2f48
0000090 e84e 2b29 08f8 0643 af7f 9824 2042 c53e
00000a0 c9fe d604                              
00000a4
  PC   Hex                   Instructions                d0       d1       d2       d3       d4       d5       d6       d7       a0       a1       a2       a3       a4       a5       a6       a7      
E 000: 0000 0000           : ori.b   #$0, D0                                                                                                                                                          
E 004: 0000 0000           : ori.b   #$0, D0    

In [17]:
program = """
dcb.l 2,0
.org $0000
	move.l  #$00410110,d2
	swap	d2
	moveq	#0,d1                   ; Set d1 to 0.
	move.w	d2,d1
	divu	#9,d1
	swap	d1
	addq.w	#1,d1	
    
    stop #$2700

"""
open("rmac_prog.s",'w').write(program)

# Assemble and run!
!./rmac/rmac -s ~oall -n -fr -rl -l -i rmac_prog.s -o tst && hexdump tst && ../../Musashi/m68kscratch/m68ksim tst

Legacy mode OFF
0000000 0000 0000 0000 0000 3c24 4100 1001 4248
0000010 0072 0232 fc82 0900 4148 4152 724e 0027
0000020
  PC   Hex                   Instructions                d0       d1       d2       d3       d4       d5       d6       d7       a0       a1       a2       a3       a4       a5       a6       a7      
E 000: 0000 0000           : ori.b   #$0, D0                                                                                                                                                          
E 004: 0000 0000           : ori.b   #$0, D0                                                                                                                                                          
E 008: 243c 0041 0110      : move.l  #$410110, D2                                                                                                                                                     
E 00e: 4842                : swap    D2                                 00410110  